In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [9]:
import string
from ipywidgets import widgets, Layout

slider_init_values = [
    ["Almond milk (14091)", 1227],
    ["Tea (14355)", 804],
    ["Cherries (9068)", 3382],
    ["Potato pancakes (11672)", 913],
]
sliders = [
    widgets.IntSlider(
        description=slider[0],
        value=slider[1],
        min=0,
        max=3500,
        style={"description_width": "200px"},
        layout=Layout(width="500px"),
    )
    for slider in slider_init_values
]

# interact won't take a list, so construct this awkward dict
slider_dict = {string.ascii_lowercase[i]: z for i, z in enumerate(sliders)}

out = widgets.interact(
    # stacked_bars,
    **slider_dict,
)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Create 4 IntSlider widgets
slider1 = widgets.IntSlider(description='Slider 1', min=0, max=100, value=50)
slider2 = widgets.IntSlider(description='Slider 2', min=0, max=100, value=50)
slider3 = widgets.IntSlider(description='Slider 3', min=0, max=100, value=50)
slider4 = widgets.IntSlider(description='Slider 4', min=0, max=100, value=50)

# Display the sliders
display(slider1, slider2, slider3, slider4)

In [6]:
from solver.load_data import load_data
from constants import NUTRIENT_NAMES, FOOD_OFFSET
import matplotlib.pyplot as plt
from ipywidgets import widgets, Layout


def stacked_bars(**kwargs):
    """
    Creates a stacked bar chart for each list in a list of lists.

    Args:
      data: A list of lists, where each sublist represents a set of values to be stacked.
      labels: A list of labels for the data sets.
    """
    colors = ["#32a852", "#53e079"]

    food_quantities = list(kwargs.values())
    food_ids = [int(k) for k in kwargs.keys()]

    # Figure out how to avoid this double-call. (Not sure if there is a way.)
    foods, max_foods, min_requirements, max_requirements = load_data(only_these_ids=food_ids)

    # Create the figure and axis
    fig, ax = plt.subplots()
    ax.invert_yaxis()
    ax.set_ylabel("Nutrients")
    ax.set_xlabel("Percentages")
    ax.set_xlim(0, 1.2)  # 0% to 120%

    for i in range(len(NUTRIENT_NAMES)):
        left = 0
        for j in range(len(foods)):
            bar_width = foods[j][FOOD_OFFSET + i] * food_quantities[j] / min_requirements[i]
            ax.barh(
                NUTRIENT_NAMES[i],
                bar_width,
                left=left,
                color=colors[j % len(colors)]
            )
            left += bar_width

    plt.tight_layout()
    plt.show()


def make_sliders(foods, max_foods):
    slider_init_values = [
        [
            foods[i][1].split(" > ")[1] + f" ({foods[i][0]})",
            max_foods[i]
        ]
        for i in range(len(foods))
    ]
    sliders = [
        widgets.IntSlider(
            description=slider[0],
            value=slider[1],
            min=0,
            max=3500,
            style={"description_width": "200px"},
            layout=Layout(width="500px"),
        )
        for slider in slider_init_values
    ]

    # widgets.interact() won't take a list, so construct this awkward dict
    slider_dict = {str(foods[i][0]): z for i, z in enumerate(sliders)}

    return slider_dict


foods, max_foods, min_requirements, max_requirements = load_data(only_these_ids=[11147, 11297])

out = widgets.interact(stacked_bars, **make_sliders(foods, max_foods))


interactive(children=(IntSlider(value=904, description='Chard, swiss, raw (11147)', layout=Layout(width='500px…